## Model Training

#### 1.1 Import Data and Required Packages
##### Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [5]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings

# Metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    AdaBoostRegressor,
    GradientBoostingRegressor
)
from sklearn.svm import SVR


# Model selection
from sklearn.model_selection import RandomizedSearchCV


#### Import the CSV Data as Pandas DataFrame

In [6]:
df = pd.read_csv('data/stud.csv')

#### Show Top 5 Records

In [9]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### Preparing X and Y variables

In [7]:
X = df.drop(columns=['math_score'],axis=1)

In [8]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [9]:
print("Categories in 'gender' variable:     ",end=" " )
print(df['gender'].unique())

print("Categories in 'race_ethnicity' variable:  ",end=" ")
print(df['race_ethnicity'].unique())

print("Categories in'parental level of education' variable:",end=" " )
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable:     ",end=" " )
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     ",end=" " )
print(df['test_preparation_course'].unique())

Categories in 'gender' variable:      ['female' 'male']
Categories in 'race_ethnicity' variable:   ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in'parental level of education' variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable:      ['standard' 'free/reduced']
Categories in 'test preparation course' variable:      ['none' 'completed']


In [10]:
y = df['math_score']

In [11]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [12]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [14]:
X = preprocessor.fit_transform(X)

In [13]:
X.shape

(1000, 7)

In [15]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

#### Create an Evaluate Function to give all metrics after model Training

In [25]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [31]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)  
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2



models = {
    "Linear Regression": Pipeline([("scaler", StandardScaler()), ("model", LinearRegression())]),
    "Lasso": Pipeline([("scaler", StandardScaler()), ("model", Lasso(max_iter=10000, random_state=42))]),
    "Ridge": Pipeline([("scaler", StandardScaler()), ("model", Ridge(random_state=42))]),
    "K-Neighbors Regressor": Pipeline([("scaler", StandardScaler()), ("model", KNeighborsRegressor())]),
   "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
   
}

rows = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_tr_pred = model.predict(X_train)
    y_te_pred = model.predict(X_test)

    tr_mae, tr_rmse, tr_r2 = evaluate_model(y_train, y_tr_pred)
    te_mae, te_rmse, te_r2 = evaluate_model(y_test, y_te_pred)

    print(name)
    print("Training  -> RMSE: {:.4f} | MAE: {:.4f} | R2: {:.4f}".format(tr_rmse, tr_mae, tr_r2))
    print("Test      -> RMSE: {:.4f} | MAE: {:.4f} | R2: {:.4f}".format(te_rmse, te_mae, te_r2))
    print("-" * 40)

    rows.append({
        "model": name,
        "train_rmse": tr_rmse, "train_mae": tr_mae, "train_r2": tr_r2,
        "test_rmse": te_rmse,   "test_mae": te_mae,  "test_r2": te_r2
    })

results = pd.DataFrame(rows).sort_values("test_rmse").reset_index(drop=True)
print("\n=== Ranked by Test RMSE (lower is better) ===")
print(results)


Linear Regression
Training  -> RMSE: 5.3340 | MAE: 4.2794 | R2: 0.8738
Test      -> RMSE: 5.4367 | MAE: 4.2357 | R2: 0.8785
----------------------------------------
Lasso
Training  -> RMSE: 5.9033 | MAE: 4.6912 | R2: 0.8454
Test      -> RMSE: 5.9106 | MAE: 4.6287 | R2: 0.8564
----------------------------------------
Ridge
Training  -> RMSE: 5.3232 | MAE: 4.2650 | R2: 0.8743
Test      -> RMSE: 5.3910 | MAE: 4.2126 | R2: 0.8806
----------------------------------------
K-Neighbors Regressor
Training  -> RMSE: 8.2382 | MAE: 6.5640 | R2: 0.6990
Test      -> RMSE: 11.2962 | MAE: 8.6910 | R2: 0.4756
----------------------------------------
Decision Tree
Training  -> RMSE: 0.2795 | MAE: 0.0187 | R2: 0.9997
Test      -> RMSE: 7.7146 | MAE: 6.1950 | R2: 0.7554
----------------------------------------
Random Forest
Training  -> RMSE: 2.3048 | MAE: 1.8325 | R2: 0.9764
Test      -> RMSE: 6.0177 | MAE: 4.6649 | R2: 0.8512
----------------------------------------
Gradient Boosting
Training  -> RMSE: 